# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False
import json
import warnings
warnings.filterwarnings("ignore")
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
clean = Data_Clean()

In [3]:
f_path="F:/myfile/p站数据/当月数据"
#f_path="F:/下班前订单数据"
#f_path="F:/订单日报/临时导出"

In [4]:
import os
excel_names = []
excel_date = []
for excel_name in os.listdir(f_path):
    # excel_names.append(excel_name)
    # excel_date.append(excel_name.split('.')[0])
    if not excel_name.startswith('~$') and excel_name.endswith('.xlsx'):
        excel_names.append(excel_name)
        excel_date.append(excel_name.split('.')[0])
print(excel_names)
print(excel_date)


['0801-0815.xlsx']
['0801-0815']


In [5]:
# df_list=[]
# for excel_name in excel_names:
#     # 读取每个excel到df
#     excel_path = f"{f_path}/{excel_name}"
#     print(excel_path)
#     df_split = pd.read_excel(excel_path,skiprows=0)
#     print(df_split)
#     df_list.append(df_split)
#     print(df_list)

# df = pd.concat(df_list,ignore_index=True)

In [6]:
df = pd.read_excel(f'F:/myfile/p站数据/当月数据/{excel_date[-1]}.xlsx')

In [7]:
df["下单日期"]=df["下单时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month

In [8]:
df = df[df['商品属性'].notnull()]

# 信息获取

In [9]:
df["拒绝理由"]=df["备注信息"].str.split("拒绝理由：").str[1]
df["拒绝理由"]=df["拒绝理由"].str.replace("[","").str.replace("]","").str.replace('"','')
df["取消原因"]=df["备注信息"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["备注信息"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
#df["无法联系原因"]=df["备注信息"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["下单商品"].str.contains('99新')|df["下单商品"].str.contains('95新')|df["下单商品"].str.contains('准新'),"二手","全新")
df.loc[:,"租赁方案"]=np.where(df["商品属性"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["预授权占用自有资金"]>0,"部分免押","全免押")
df.loc[:,"是否使用优惠券"]=np.where((df["设置买断金额"]>df["总租金"])&(df["租赁方案"]=="租完即送"),"使用优惠券","未使用优惠券")
df.loc[:,"是否进行预授权"]=np.where(df["预授权占用自有资金"].isnull(),"未预授权","已预授权")

In [10]:
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["商品属性"]),axis=1)

In [11]:
def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["商品属性"]),axis=1)

In [12]:
df.loc[:,"进件"]=np.where((df["订单状态"]=="待支付")|(df["订单状态"]=="订单取消"),"未进件","进件")

# 渠道归属

In [13]:
df.loc[:,"来源渠道"]=df["来源渠道"].fillna("未知渠道")

In [14]:
def qudao_type(a,b):
    a = str(a)
    b = str(b)
    if '京东活动' in b:
        return '京享租'
    elif "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "支付宝直播" in a or '支付宝直播' in b or '支付宝直播商品' in b:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in b:
        return "生活号"
    elif "社群" in b or '支群' in b:
        return "支付宝社群"
    
    else :
        return a

In [15]:
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["活动名称"]),axis=1)

In [16]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["订单状态"].map(dict_status_code)

In [17]:
df.sort_values(by=["下单日期","状态编码"],inplace=True)

# 删除重复数据

In [18]:
#删除重复单号
df.drop_duplicates(subset=["单号"],inplace=True)
#删除身份证空值行
df.dropna(subset=["下单用户身份证号"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['备注信息'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['活动名称']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['活动名称']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["订单状态"],axis=0,inplace=True)
df.drop(df[df['下单用户'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华","邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文","王家伟"])].index,inplace=True)

In [19]:
#删除重复订单
df.drop_duplicates(subset=["单号"],inplace=True)
df.drop_duplicates(subset=["下单用户","下单手机","下单用户身份证号","下单日期"],keep="last",inplace=True)

# 定义状态

In [20]:
def reject_type(a,b,c,d,e,f,g):
    if a is not np.nan and b=="未进件":
        return "前置拦截"
    elif g is np.nan and b=="未进件":
        return "进件前取消"
    elif a is not np.nan and b=="进件":
        return "机审强拒"
    elif c is not np.nan and"抖音非免押用户" in c:
        return "抖音非免押拒绝"
    elif a is np.nan and b=="进件" and c is not np.nan and '已退款' in e:
        return "人审拒绝"
    elif a is np.nan and b=="进件" and d is not np.nan:
        return "客户取消"
    #elif a is np.nan and b=="进件" and f is not np.nan:
        #return "无法联系"
    elif g is not np.nan and b=="进件" and "蚂蚁数控风险等级" in g:
        return "出库前风控强拒"
    elif g is not np.nan and b=="进件" and "命中出库前风控流强拒" in g:
        return "出库前风控强拒"
    elif a is np.nan and b=="进件" and "待审核" in e:
        return "待审核"
    elif a is np.nan and b=="进件" and "待发货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "待收货" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "租赁中" in e:
        return "出库"
    elif a is np.nan and b=="进件" and "已完成" in e:
        return "出库"
    elif a is np.nan and b=="未进件":
        return "未进件"
    else :
        return e
    
    

In [21]:
df.loc[:,"审核状态"]=df.apply(lambda x: reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["订单状态"],x["订单状态"],x["备注信息"]),axis=1)

In [22]:

fw_path = 'F:/myfile/p站数据/服务订单'
fw_excel_names = []
fw_excel_date = []
for excel_name in os.listdir(fw_path):
    fw_excel_names.append(excel_name)
    fw_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(fw_excel_date)


['~$服务订单0611-0617', '服务订单0801-0815']


In [23]:
f_path_insure=f"F:/myfile/p站数据/服务订单/{fw_excel_date[-1]}.xlsx"
df_insure=pd.read_excel(f_path_insure,skiprows=0)
df_insure1=pd.merge(df,df_insure,left_on="单号", right_on="租赁订单号",how="left")
df=df_insure1

In [24]:
import datetime

Today=datetime.date.today()
str(Today)
Now=str(datetime.datetime.now() )
Now

'2025-08-21 15:25:19.880223'

# 删除商家数据

In [25]:
df_contain = df.copy()

In [26]:
df.drop(df[df['商家']=="深圳优优大数据科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="优优2店"].index,inplace=True)
df.drop(df[df['商家']=="探路者自营账户"].index,inplace=True)
df.drop(df[df['商家']=="苏州蚁诺宝"].index,inplace=True)
df.drop(df[df['商家']=="租着用电脑数码"].index,inplace=True)
df.drop(df[df['商家']=="北京海鸟窝科技有限公司"].index,inplace=True)
df.drop(df[df['商家']=="澄心优租"].index,inplace=True)
df.drop(df[df['商品型号'].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
# 趣智数码  单
df.drop(df[df['商家']=="趣智数码"].index,inplace=True)
df.drop(df[df['商家']=="格木木二奢名品"].index,inplace=True)
df.drop(df[df['商家']=="广州康基贸易有限公司"].index,inplace=True)
df.drop(df[df['商家']=="汇客好租"].index,inplace=True)
df.drop(df[df['商家']=="线下小店"].index,inplace=True)
df.drop(df[df['商家']=="乙辉数码"].index,inplace=True)
df.drop(df[df['商家']=="兴鑫兴通讯"].index,inplace=True)
df.drop(df[df['商家']=="南京聚格网络科技"].index,inplace=True)
df.drop(df[df['商家']=="呱子笔记本电脑"].index,inplace=True)

df.drop(df[df['商家'] == "星晟数码"].index, inplace=True)
df.drop(df[df['商家'] == "蘑菇时间"].index, inplace=True)
# df = clean.drop_merchant(df)

In [27]:
df_j=df[df["进件"]=="进件"]

In [28]:
# 商家拒量
df.drop(df[df['商家']=="小蚂蚁租机"].index,inplace=True)
df.drop(df[df['商家']=="兴鑫兴通讯"].index,inplace=True)
df.drop(df[df['商家']=="人人享租"].index,inplace=True)
df.drop(df[df['商家']=="崇胜数码"].index,inplace=True)
df.drop(df[df['商家']=="喜卓灵租机"].index,inplace=True)
df.drop(df[df['商家']=="喜卓灵薪租机"].index,inplace=True)

In [29]:
df_j[df_j.归属渠道=='搜索渠道'].shape

(5301, 88)

# 以下为出库相关数据，要统计到出库相关的数据，必须运行到以下读取出库台账的代码

In [30]:
#读取导入出库台账数据
# f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# df_ck = pd.read_excel(f_path_ck,sheet_name="2023",skiprows=0) 
f_path_ck="F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck=pd.read_excel(f_path_ck,sheet_name="2025",skiprows=0)
dfck=pd.merge(df_ck,df,left_on="订单号", right_on="单号")
dfck.drop_duplicates(subset=["订单号"],inplace=True)


In [31]:
dfck.订单状态.value_counts()

订单状态
租赁中          2662
待收货           227
已退款            85
已完成            63
待发货            16
检测中             2
待商家收货/归还中       1
Name: count, dtype: int64

# 检查出库订单中有没有客户取消的单，即订单状态为”已退款"的单，找出来删除后重新运行上一块导入出库台账的代码

In [32]:
dfck.drop(dfck[dfck["订单状态"]=="已退款"].index,inplace=True)

# 剔除拒收!!!!
dfck.drop(dfck[dfck["订单状态"]=="已完成"].index,inplace=True)

In [33]:
dfck.订单状态.value_counts()

订单状态
租赁中          2662
待收货           227
待发货            16
检测中             2
待商家收货/归还中       1
Name: count, dtype: int64

In [34]:
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)

# 苹果15系列商品型号-进件

In [35]:
# #苹果15系列机型进件
# df_j["型号内存"]=df_j["商品型号"]+"_"+df_j["内存"]
# df_jap15=df_j[df_j["型号内存"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_jap15["下单日期"],df_jap15["型号内存"],margins=True).to_excel(f"F:苹果15进件.xlsx")

In [36]:
# #华为进件
# df_jhw=df_j[df_j["商品型号"].str.contains(pat="Mate60",regex=False)==True]
# pd.crosstab(df_jhw["商品型号"],df_jhw["内存"],margins=True)


In [37]:
# df_jhw.groupby(["商品型号","内存","颜色"]).agg({"单号":np.size}).unstack(-1)

# 苹果15系列商品型号-出库

In [38]:
# # 苹果15系列商品型号-出库
# df_ap15=dfck[dfck["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_ap15["下单日期"],df_ap15["商品型号"],margins=True)

In [39]:
# #华为出库
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60",regex=False)==True]

# pd.crosstab(dfckhw["商品型号"],dfckhw["内存_x"],margins=True)

In [40]:
# dfckhw.groupby(["商品型号","内存_y","颜色_x"]).agg({"订单号":np.size}).unstack(-1)

# 某渠道苹果15系列商品型号-进件

In [41]:
#苹果15系列商品型号-去重订单
# df_qd=df_j[df_j["来源渠道"]=="芝麻租物"]
# df_jap15=df_qd[df_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_jap15["下单日期"],df_jap15["商品型号"],margins=True).to_excel("F:/芝麻租物/2024.3芝麻渠道15进件.xlsx")

# 某渠道苹果15系列商品型号-去重订单


In [42]:
# # 某渠道苹果15系列商品型号-进件
# df_qd=df[df["来源渠道"]=="芝麻租物"]
# df_ap15=df_qd[df_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(df_ap15["下单日期"],df_ap15["商品型号"],margins=True).to_excel("F:/线上/芝麻租物/2024.3芝麻渠道15去重订单.xlsx")

# 某渠道苹果15系列商品型号-出库

In [43]:
# # 某渠道苹果15系列商品型号-出库
# dfck_qd=dfck[dfck["来源渠道"]=="芝麻租物"]
# dfck_jap15=dfck_qd[dfck_qd["商品型号"].str.contains(pat="15",regex=False)==True]
# pd.crosstab(dfck_jap15["下单日期"],dfck_jap15["商品型号"],margins=True).to_excel("F:/线上/芝麻租物/2024.3芝麻渠道15出库.xlsx")

# 宏生碎屏险数据，导出来直接甩给他就可以了

In [44]:
# #总体出库订单碎屏险购买数据
# df_bx_ck=pd.crosstab(dfck["下单日期"],dfck["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.drop(columns={"All"})
# df_s2merge=pd.merge(df_bx_ck,df_s3,left_index=True, right_index=True)
# df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"] #+df_s2merge["待支付"]
# df_s2merge["实际支付碎屏险比例"]=df_s2merge["最终实际支付碎屏险出库"]/df_s2merge["合计出库"]
# df_s2merge["实际支付碎屏险比例"]=df_s2merge["实际支付碎屏险比例"].apply(lambda x:format(x,".2%"))
# df_s2merge

In [45]:
# #芝麻租物出库订单碎屏险购买数据
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zw3=pd.crosstab(dfckzw["下单日期"],dfckzw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]+df_zws2merge["待确认"]#+df_s2merge["待支付"]
# df_zws2merge

In [46]:
# #Mate60Pro 购买碎屏险数据
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60Pro",regex=False)==True]
# df_zw3=pd.crosstab(dfckhw["下单日期"],dfckhw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckhw["下单日期"],dfckhw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]#+df_zws2merge["待确认"]
# df_zws2merge

In [47]:
# #Mate60 购买碎屏险数据
# dfckhw=dfck[dfck["商品型号"].str.contains(pat="Mate60",regex=False)==True]
# df_zw3=pd.crosstab(dfckhw["下单日期"],dfckhw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zws3=pd.crosstab(dfckhw["下单日期"],dfckhw["服务订单状态"],margins=True)
# df_zws3=df_zws3.drop(columns={"All"})
# df_zws2merge=pd.merge(df_zw3,df_zws3,left_index=True, right_index=True)
# # df_zws2merge["最终实际支付碎屏险出库"]=df_zws2merge["服务中"]   # df_zws2merge["待确认"]+
# df_zws2merge

# 总体碎屏险数据

In [48]:
# #总体碎屏险数据,宏生需求
# df_bx=pd.crosstab(df["下单日期"],df["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx=df_bx.rename(columns={1:"不购买碎屏险订单",2:"购买碎屏险订单","All":"合计下单",})
# df_bx_j=pd.crosstab(df_j["下单日期"],df_j["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_j=df_bx_j.rename(columns={1:"不购买碎屏险进件",2:"购买碎屏险进件","All":"合计进件",})
# df_bx_ck=pd.crosstab(dfck["下单日期"],dfck["是否购买服务商品：1、否；2、是"],margins=True)
# df_bx_ck=df_bx_ck.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库",})
# df_spx1=pd.merge(df_bx,df_bx_j,left_index=True, right_index=True)
# df_spx2=pd.merge(df_spx1,df_bx_ck,left_index=True, right_index=True)#.to_excel(f"F:/碎屏险/总体碎屏险数据{excel_date[-1]}.xlsx")

In [49]:
# df_s1=pd.crosstab(df["下单日期"],df["服务订单状态"],margins=True)
# df_s1=df_s1.rename(columns={"All":"合计购买碎屏险订单"})
# df_s2=pd.crosstab(df_j["下单日期"],df_j["服务订单状态"],margins=True)
# df_s2=df_s2.rename(columns={"All":"合计购买碎屏险进件"})
# df_s3=pd.crosstab(dfck["下单日期"],dfck["服务订单状态"],margins=True)
# df_s3=df_s3.rename(columns={"All":"合计购买碎屏险出库"})
# df_s1merge=pd.merge(df_s1,df_s2,left_index=True, right_index=True)
# df_s2merge=pd.merge(df_s1merge,df_s3,left_index=True, right_index=True)
# df_s2merge["最终实际支付碎屏险出库"]=df_s2merge["待确认"]+df_s2merge["服务中"]
# df_s2merge

# 芝麻租物碎屏险数据

In [50]:
# #芝麻租物碎屏险数据,宏生需求
# dfzw=df[df["归属渠道"]=="芝麻租物"]
# df_zw1=pd.crosstab(dfzw["下单日期"],dfzw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw1=df_zw1.rename(columns={1:"不购买碎屏险订单",2:"购买碎屏险订单","All":"合计下单"})

# df_j_zw=df_j[df_j["归属渠道"]=="芝麻租物"]
# df_zw2=pd.crosstab(df_j_zw["下单日期"],df_j_zw["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw2=df_zw2.rename(columns={1:"不购买碎屏险进件",2:"购买碎屏险进件","All":"合计进件"})
# df2=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zw3=pd.crosstab(df2["下单日期"],df2["是否购买服务商品：1、否；2、是"],margins=True)
# df_zw3=df_zw3.rename(columns={1:"不购买碎屏险出库",2:"购买碎屏险出库","All":"合计出库"})
# df_zwspx1=pd.merge(df_zw1,df_zw2,left_index=True, right_index=True)
# df_zwspx2=pd.merge(df_zwspx1,df_zw3,left_index=True, right_index=True)#.to_excel(f"F:/碎屏险/芝麻租物碎屏险数据{excel_date[-1]}.xlsx")

In [51]:
# dfzw=df[df["归属渠道"]=="芝麻租物"]
# df_zws1=pd.crosstab(dfzw["下单日期"],dfzw["服务订单状态"],margins=True)
# df_zws1=df_zws1.rename(columns={"All":"合计购买碎屏险订单"})
# df_j_zw=df_j[df_j["归属渠道"]=="芝麻租物"]
# df_zws2=pd.crosstab(df_j_zw["下单日期"],df_j_zw["服务订单状态"],margins=True)
# df_zws2=df_zws2.rename(columns={"All":"合计购买碎屏险进件"})
# dfckzw=dfck[dfck["归属渠道"]=="芝麻租物"]
# df_zws3=pd.crosstab(dfckzw["下单日期"],dfckzw["服务订单状态"],margins=True)
# df_zws3=df_zws3.rename(columns={"All":"合计购买碎屏险出库"})
# df_zws1merge=pd.merge(df_zws1,df_zws2,left_index=True, right_index=True)
# df_zws2merge=pd.merge(df_zws1merge,df_zws3,left_index=True, right_index=True)
# df_zws2merge["最终实际支付碎屏险出库"]=+df_zws2merge["服务中"]+df_zws2merge["待确认"]
# df_zws2merge

In [52]:
#dfck[dfck["是否购买服务商品：1、否；2、是"]==2].to_excel(f"C:/Users/Administrator/Desktop/总体碎屏险数据出库_1119.xlsx")

# 月中\月底运营数据复盘 

# 月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比

In [53]:
#月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck.groupby(["月份","来源渠道"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"总押金":np.mean,}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["总押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,来源渠道,出库,总租金均值,签约价均值,总押金均值,租售比
0,8,京享租右卡,16,6444.538750,12612.462500,9874.000000,65.27%
1,8,八派信息,8,13321.775000,13321.775000,12447.512500,107.02%
2,8,其他渠道场景渠道。,12,8237.381667,13196.666667,10415.475000,79.09%
3,8,单人聊天会话中的小程序消息卡片（分享）,3,9676.393333,13863.566667,10665.666667,90.72%
4,8,我的小程序入口,6,7441.673333,12996.700000,9999.000000,74.42%
5,8,搜索-租手机,41,8301.512439,13060.226829,10354.975610,80.17%
6,8,搜索-租苹果手机,20,7933.532000,13191.670000,10298.885000,77.03%
7,8,搜索下挂-商品,204,7692.606618,13117.828431,10187.214706,75.51%
8,8,搜索下挂券,1,12997.700000,12997.700000,9999.000000,129.99%
9,8,搜索直达banner,1,5865.570000,10397.100000,7999.000000,73.33%


In [54]:
#月中/月底各渠道出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck.groupby(["月份","归属渠道"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"总押金":np.mean,}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["总押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,归属渠道,出库,总租金均值,签约价均值,总押金均值,租售比
0,8,京享租,745,6443.468228,12621.495570,9790.677852,65.81%
1,8,八派信息,8,13321.775000,13321.775000,12447.512500,107.02%
2,8,其他渠道场景渠道。,10,8724.813000,13256.660000,10498.770000,83.10%
3,8,单人聊天会话中的小程序消息卡片（分享）,3,9676.393333,13863.566667,10665.666667,90.72%
4,8,我的小程序入口,5,7611.584000,13256.660000,10199.000000,74.63%
5,8,抖音渠道,1,4896.000000,9599.000000,7999.000000,61.21%
6,8,搜索渠道,545,7886.923670,13011.343119,10182.997248,77.45%
7,8,支付宝 push 消息（同1014）,1,9097.300000,9097.300000,9097.300000,100.00%
8,8,支付宝客户端首页,18,7367.671667,11906.416667,9443.277778,78.02%
9,8,支付宝直播,3,5866.580000,12996.700000,9999.000000,58.67%


# 月中/月底总体出库单数、租金均值、签约价均值、押金均值、租售比

In [55]:
dfck_all=dfck.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,8,2908,6506.995076,12796.39412,9909.660591,9935.328645,65.66%


In [56]:
#月中/月底总体出库单数、租金均值、签约价均值、押金均值、租售比
dfck_fjd = dfck[dfck.归属渠道!='京享租']
dfck_all=dfck_fjd.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,8,2163,6528.875566,12856.634258,9950.641701,9985.150578,65.61%


# 京享租

In [57]:
dfck_jd = dfck[dfck.归属渠道=='京享租']
dfck_all=dfck_jd.groupby(["月份"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,8,745,6443.468228,12621.49557,9790.677852,9790.677852,65.81%


# 月中/月底各租赁方案出库单数、租金均值、签约价均值、押金均值、租售比

In [58]:
#月中/月底各租赁方案出库单数、租金均值、签约价均值、押金均值、租售比
dfck_all=dfck_fjd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,租赁方案,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,8,租完即送,90,12925.162222,12925.162222,10499.000000,11328.363333,123.11%
1,8,租完归还,2073,6251.178606,12853.659093,9926.834539,9926.834539,62.97%


# 京享租

In [59]:
dfck_all=dfck_jd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"总租金":np.mean,"买断金额":np.mean,"商品押金":np.mean,"总押金":np.mean}).reset_index()
dfck_all=dfck_all.rename(columns={"月份":"月份","订单号":"出库","总租金":"总租金均值","买断金额":"签约价均值","商品押金":"商品押金均值","总押金":"总押金均值"})
dfck_all["租售比"]=dfck_all["总租金均值"]/dfck_all["商品押金均值"]
dfck_all["租售比"]=dfck_all["租售比"].apply(lambda x:format(x,".2%"))
dfck_all

,月份,租赁方案,出库,总租金均值,签约价均值,商品押金均值,总押金均值,租售比
0,8,租完归还,745,6443.468228,12621.49557,9790.677852,9790.677852,65.81%


# 月中月底各渠道各租赁方案出库数

In [60]:
#月中月底各渠道租赁方案出库数
pd.crosstab(dfck["来源渠道"],dfck["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
来源渠道,,,
京享租右卡,0,16,16
八派信息,8,0,8
其他渠道场景渠道。,2,10,12
单人聊天会话中的小程序消息卡片（分享）,1,2,3
我的小程序入口,0,6,6
搜索-租手机,6,35,41
搜索-租苹果手机,1,19,20
搜索下挂-商品,12,192,204
搜索下挂券,1,0,1


In [61]:
#月中月底各渠道租赁方案出库数
pd.crosstab(dfck["归属渠道"],dfck["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
归属渠道,,,
京享租,0,745,745
八派信息,8,0,8
其他渠道场景渠道。,2,8,10
单人聊天会话中的小程序消息卡片（分享）,1,2,3
我的小程序入口,0,5,5
抖音渠道,0,1,1
搜索渠道,47,498,545
支付宝 push 消息（同1014）,1,0,1
支付宝客户端首页,2,16,18


# 月中月底各渠道租赁方案进件数

In [62]:
#月中月底各渠道租赁方案进件数
pd.crosstab(df_j["来源渠道"],df_j["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
来源渠道,,,
九州信息,6,0,6
京享租右卡,0,1568,1568
付费流量(通过商家数字推广平台，灯火等投放的广告),2,13,15
侠客行全域租物,1,2,3
八派信息,429,22,451
其他渠道场景渠道。,16,82,98
单人聊天会话中的小程序消息卡片（分享）,17,1060,1077
宜品花,20,2,22
小程序商家消息（服务提醒）,3,34,37


In [63]:
df_zulin = pd.crosstab(df_j["来源渠道"],df_j["租赁方案"],margins=True)
# df_zulin.to_excel(f'F:/各渠道租赁进件数/{excel_date[-1]}进件数.xlsx')

In [64]:
#月中月底各渠道租赁方案进件数
pd.crosstab(df_j["归属渠道"],df_j["租赁方案"],margins=True)

租赁方案,租完即送,租完归还,All
归属渠道,,,
九州信息,6,0,6
京享租,1,17783,17784
京享租右卡,0,1,1
付费流量(通过商家数字推广平台，灯火等投放的广告),2,8,10
八派信息,429,18,447
其他渠道场景渠道。,15,66,81
单人聊天会话中的小程序消息卡片（分享）,17,1028,1045
宜品花,20,2,22
小程序商家消息（服务提醒）,3,19,22


# 以下代码为测算毛利时候使用，使用前先从M站系统导出出库单获取成本数据，然后按顺序运行以下代码

In [65]:

ck_path = 'F:/myfile/p站数据/出库单'
ck_excel_names = []
ck_excel_date = []
for excel_name in os.listdir(ck_path):
    if not excel_name.startswith('~$') and excel_name.endswith('.xlsx'):
        ck_excel_names.append(excel_name)
        ck_excel_date.append(excel_name.split('.')[0])
# print(excel_names)
print(ck_excel_date)
os.listdir(ck_path)

['8月出库']


['8月出库.xlsx', '~$7月出库.xlsx']

In [66]:
f_path_cost=f"F:/myfile/p站数据/出库单/{ck_excel_date[-1]}.xlsx"
df_cost=pd.read_excel(f_path_cost,sheet_name='出库表',skiprows=0)
#df_cost.loc[:,"成本"]=np.where(df_cost["供应商"].str.contains('侯立')|df_cost["供应商"].str.contains('张陵')|df_cost["供应商"].str.contains('拾贝壳')|df_cost["供应商"].str.contains('大龙发'),(df_cost["设备成本"]-10)/1.025,df_cost["设备成本"])
# df_cost["成本"]=df_cost["设备成本"]
# df_cost["成本"]=df_cost["成本"].round()
dfck_cost=pd.merge(dfck,df_cost,left_on="订单号", right_on="订单号")
# 更改原因：由于台账存在成本字段与出库单表的成本字段发生冲突
dfck_cost.loc[:, "成本"]=np.where(dfck_cost["设备成本"] is np.nan, dfck_cost["成本"],dfck_cost["设备成本"])
dfck_cost["成本"]=dfck_cost["成本"].round()
dfck_cost.drop_duplicates(subset=["订单号"],inplace=True)
# 宏生需求
dfck_cost.to_excel(f"F:/毛利/明细_{Today}.xlsx")

In [67]:
dfck_cost.columns

Index(['日期', '订单号', '类型', '型号', '内存_x', '颜色_x', '平台套餐', '租金', '已付租金', '期数',
       ...
       '仓库名称', '设备成本', '设备零售价', '成新度', '设备串码_y', 'SN码', '出库原因', '操作人', '出库数量',
       '出库时间'],
      dtype='object', length=121)

In [68]:
# 毛利计算过程，这块代码不能忽略
dfck_cost.loc[:,"原始毛利"]=dfck_cost["买断金额"]-dfck_cost["成本"]+dfck_cost["商品溢价费"]
# dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),"苹果","非苹果")
dfck_cost["商品型号_all"] = dfck_cost["商品型号"][dfck_cost["商品型号"].str.contains('iP')==True] 
# dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),  
#                                 np.where(dfck_cost["商品型号_all"].str.contains('16'), "苹果16", "苹果15"),"非苹果")
dfck_cost.loc[:,"残值机型"]=np.where(dfck_cost["商品型号"].str.contains('iP'),  
                                np.where(dfck_cost["商品型号_all"].str.contains('16'), "苹果16",
                                        np.where(dfck_cost['商品型号_all'].str.contains('15'), "苹果15",
                                                np.where(dfck_cost['商品型号_all'].str.contains('14'), "苹果14", '苹果14以下'))),"非苹果")
def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果16":
        return s["成本"]*0.6
    elif s["残值机型"]=="苹果15":
        return s["成本"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["成本"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["成本"]*0.375
    else:
        return s["成本"]*0.425
dfck_cost.loc[:,"残值"]=dfck_cost.apply(getrecanzhi,axis=1)
dfck_cost.loc[:,"总应收"]=dfck_cost["总租金"]+dfck_cost["残值"]+dfck_cost["商品溢价费"]
dfck_cost.loc[:,"残值估算毛利"]=dfck_cost["总应收"]-dfck_cost["成本"]
def getremark(s):
    if s["成本"]==0:
        return s["已付金额"]
    else:
        return s["残值估算毛利"]
dfck_cost["残值估算毛利"]=dfck_cost.apply(getremark,axis=1)
dfck2 = dfck_cost[dfck_cost["成本"].notnull()]

In [69]:
# dfck_cost.groupby('月份').agg({'订单号': 'count', '总租金': 'sum', '残值': 'sum', '商品溢价费': 'sum', '总应收': 'sum', '成本': 'sum', '残值估算毛利': 'mean', '买断金额': 'sum', '原始毛利': 'mean'})

# 月中、月底总体数据测算毛利

In [70]:
# 月中、月底总体数据测算毛利
dfck2_fjd = dfck2[dfck2.归属渠道!='京享租']
dfck_zsb=dfck2_fjd.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,订单号,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,8,2154,8482.667131,12855.53844,4372.871309,2916.296219,34.38%,51.55%


In [71]:
dfck2_jd = dfck2[dfck2.归属渠道=='京享租']
dfck_zsb=dfck2_jd.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,订单号,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,8,738,8365.52168,12629.878455,4264.356775,3084.323991,36.87%,50.98%


# 租物和非租物

In [72]:
# # 月中、月底总体数据测算毛利
# dfck2_zw = dfck2[dfck2.归属渠道=='芝麻租物']
# dfck_zsb=dfck2_zw.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb


In [73]:
# dfck2_fzw = dfck2[dfck2.归属渠道!='芝麻租物']
# dfck_zsb=dfck2_fzw.groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
# dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb.rename(columns={'月份':'月份','单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
# dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
# dfck_zsb

# 月中、月底分租赁方案测算毛利

In [74]:
dfck_zsb=dfck2.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,8,租完即送,90,8509.788889,12925.162222,4415.373333,4415.373333,51.89%,51.89%
1,8,租完归还,2802,8450.941827,12793.867059,4342.925232,2912.401699,34.46%,51.39%


In [75]:
#月中、月底分租赁方案测算毛利
dfck_zsb=dfck2_fjd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,8,租完即送,90,8509.788889,12925.162222,4415.373333,4415.373333,51.89%,51.89%
1,8,租完归还,2064,8481.484496,12852.502519,4371.018023,2850.929484,33.61%,51.54%


# 京享租

In [76]:
dfck_zsb=dfck2_jd.groupby(["月份","租赁方案"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,租赁方案,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,8,租完归还,738,8365.52168,12629.878455,4264.356775,3084.323991,36.87%,50.98%


# 月中、月底分渠道测算毛利

In [77]:
# 月中、月底分渠道测算毛利
dfck_zsb=dfck2.groupby(["月份","来源渠道"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,来源渠道,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,8,京享租右卡,15,8241.0,12593.380000,4352.38,3078.741333,37.36%,52.81%
1,8,八派信息,8,8862.875,13321.775000,4458.9,4458.900000,50.31%,50.31%
2,8,其他渠道场景渠道。,12,8765.833333,13196.666667,4430.833333,3764.048333,42.94%,50.55%
3,8,单人聊天会话中的小程序消息卡片（分享）,3,9301.0,13863.566667,4562.566667,4233.993333,45.52%,49.05%
4,8,我的小程序入口,6,8638.666667,12996.700000,4358.033333,3986.206667,46.14%,50.45%
5,8,搜索-租手机,41,8613.804878,13060.226829,4446.421951,4076.136829,47.32%,51.62%
6,8,搜索-租苹果手机,20,8764.5,13191.670000,4427.17,4174.832000,47.63%,50.51%
7,8,搜索下挂-商品,204,8650.014706,13117.828431,4467.813725,3937.562500,45.52%,51.65%
8,8,搜索下挂券,1,8440.0,12997.700000,4557.7,4557.700000,54.00%,54.00%
9,8,搜索直达banner,1,7052.0,10397.100000,3345.1,3044.770000,43.18%,47.43%


In [78]:
dfck_zsb=dfck2.groupby(["月份","归属渠道"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,归属渠道,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,8,京享租,738,8365.52168,12629.878455,4264.356775,3084.323991,36.87%,50.98%
1,8,八派信息,8,8862.875,13321.775000,4458.9,4458.900000,50.31%,50.31%
2,8,其他渠道场景渠道。,10,8803.0,13256.660000,4453.66,4043.213000,45.93%,50.59%
3,8,单人聊天会话中的小程序消息卡片（分享）,3,9301.0,13863.566667,4562.566667,4233.993333,45.52%,49.05%
4,8,我的小程序入口,5,8818.4,13256.660000,4438.26,4084.224000,46.31%,50.33%
5,8,抖音渠道,1,3830.0,9599.000000,5769.0,2866.100000,74.83%,150.63%
6,8,搜索渠道,543,8591.38674,13011.397053,4420.010313,3997.537735,46.53%,51.45%
7,8,支付宝 push 消息（同1014）,1,6046.0,9097.300000,3051.3,3051.300000,50.47%,50.47%
8,8,支付宝客户端首页,18,7848.611111,11906.416667,4057.805556,3675.653611,46.83%,51.70%
9,8,支付宝直播,3,8563.333333,12996.700000,4433.366667,2441.246667,28.51%,51.77%


In [79]:
dfck_zsb =dfck2[dfck2["归属渠道"]=="搜索渠道"].groupby(["月份"]).agg({"订单号":np.size,"成本":np.mean,"买断金额":np.mean,"原始毛利":np.mean,"残值估算毛利":np.mean,}).reset_index()
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利"]/dfck_zsb["成本"]
dfck_zsb["残值估算毛利率"]=dfck_zsb["残值估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb.rename(columns={'月份':'月份','订单号':'出库','成本':'采购成本均值','买断金额':'签约价均值','原始毛利':'买断估算毛利'},inplace=True)
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利"]/dfck_zsb["采购成本均值"]
dfck_zsb["买断估算毛利率"]=dfck_zsb["买断估算毛利率"].apply(lambda x:format(x,".2%"))
dfck_zsb

,月份,出库,采购成本均值,签约价均值,买断估算毛利,残值估算毛利,残值估算毛利率,买断估算毛利率
0,8,543,8591.38674,13011.397053,4420.010313,3997.537735,46.53%,51.45%


# 商家数据

In [80]:
merchant_list = ['澄心优租', '优优2店', '北京海鸟窝科技有限公司', '租着用电脑数码', '趣智数码', '汇客好租', '乙辉数码','小蚂蚁租机', '兴鑫兴通讯', '呱子笔记本电脑', '南京聚格网络科技', '人人享租', '喜卓灵租机', '喜卓灵新租机', '星晟数码', '蘑菇时间']
df_merchant = df_contain[df_contain.商家.isin(merchant_list)]
df_merchant.loc[:, '是否进件'] = np.where(df_merchant.进件=='进件', 1, 0)
df_merchant.loc[:, '出库'] = np.where(df_merchant.审核状态=='出库', 1, 0)
df_merchant_group = df_merchant.groupby(['月份', '商家']).agg({'单号': 'count', '是否进件': 'sum', '出库': 'sum', '总租金': 'mean', '买断金额': 'mean'}).rename(columns={'单号': '去重订单数', '是否进件': '进件数', '总租金': '租金平均价', '买断金额': '买断平均价'})
df_merchant_group.loc[:, '进件出库率'] = (df_merchant_group.出库/df_merchant_group.进件数).map(lambda x: format(x,".2%"))
df_merchant_group = df_merchant_group.reset_index()
df_merchant_group[['月份', '商家', '去重订单数', '进件数', '出库', '进件出库率', '租金平均价', '买断平均价']]

,月份,商家,去重订单数,进件数,出库,进件出库率,租金平均价,买断平均价
0,8,人人享租,2767,2756,105,3.81%,7943.332830,13279.986484
1,8,北京海鸟窝科技有限公司,5,1,0,0.00%,760.200000,3980.000000
2,8,南京聚格网络科技,2,1,0,0.00%,360.000000,6500.000000
3,8,喜卓灵新租机,1286,1284,32,2.49%,7613.009689,13524.317652
4,8,星晟数码,752,241,14,5.81%,4391.150718,8463.177793
5,8,汇客好租,1232,523,146,27.92%,4933.015211,9888.008117
6,8,澄心优租,34,8,1,12.50%,3778.910882,7051.176471
7,8,租着用电脑数码,199,85,9,10.59%,1323.532563,4127.391960
8,8,蘑菇时间,45,13,5,38.46%,482.575111,4447.788889
9,8,趣智数码,36,12,2,16.67%,478.111111,4716.916667


# 机型均值

In [81]:
# # 商品押金  签约价   采购价   
dfck2["型号内存"]=dfck2["商品型号"]+"_"+dfck2["内存_x"]
dfck2.groupby(['型号内存']).agg({'商品押金':'mean','买断金额':'mean','成本':'mean'})

,商品押金,买断金额,成本
型号内存,,,
Airpods Pro（第二代）_/,1899.000000,2468.300000,1445.0
Apple Watch Ultra 2 2024款_GPS+蜂窝款49mm,6499.000000,8447.400000,5299.0
HUAWEI Pura 80 Pro_12GB+512G,6999.000000,9097.300000,5863.0
MacBook Air_13英寸M4【10核+8核】16GB+256GB,7999.000000,10397.100000,6320.0
iPad Air 2025款 11英寸_128GB WiFi版,4799.000000,6237.700000,3874.0
iPad Pro 2024款_256G WiFi版,8999.000000,11696.900000,7385.0
iPad11 2025款 11英寸_256GB WiFi版,3499.000000,4548.000000,3023.0
iPhone 12_128G,2400.000000,3120.100000,1814.0
iPhone 16e_128GB,4499.000000,5847.800000,4274.0


In [82]:
# df_zulin = dfck2.groupby(['型号内存']).agg({'订单号':'count', '商品押金':'mean','买断金额':'mean','成本':'mean'})
# df_zulin.to_excel(f'F:/宏生/{excel_date[-1]}宏生转化金额.xlsx')